<a href="https://colab.research.google.com/github/hananbahtiti/Hybrid-Intrusion-detection-Systems/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#test

In [ ]:
!wget -O NGIDS-DS.rar https://unsworks.unsw.edu.au/bitstreams/0ac2820a-5131-43ab-90b2-c624c8d73649/download

--2025-04-19 05:33:36--  https://unsworks.unsw.edu.au/bitstreams/0ac2820a-5131-43ab-90b2-c624c8d73649/download
Resolving unsworks.unsw.edu.au (unsworks.unsw.edu.au)... 54.253.215.118
Connecting to unsworks.unsw.edu.au (unsworks.unsw.edu.au)|54.253.215.118|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://unsworks.unsw.edu.au/server/api/core/bitstreams/0ac2820a-5131-43ab-90b2-c624c8d73649/content [following]
--2025-04-19 05:33:37--  https://unsworks.unsw.edu.au/server/api/core/bitstreams/0ac2820a-5131-43ab-90b2-c624c8d73649/content
Reusing existing connection to unsworks.unsw.edu.au:443.
HTTP request sent, awaiting response... 200 200
Length: unspecified [application/x-rar-compressed]
Saving to: ‘NGIDS-DS.rar’

NGIDS-DS.rar            [ <=>                ] 941.51M  16.4MB/s    in 57s     

2025-04-19 05:34:35 (16.4 MB/s) - ‘NGIDS-DS.rar’ saved [987249484]



In [ ]:
!unrar x /content/NGIDS-DS.rar



UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/NGIDS-DS.rar


Would you like to replace the existing file NGIDS/NGIDS-DS-v1/feature_descr.csv
  2286 bytes, modified on 2016-07-21 10:09
with a new one
  2286 bytes, modified on 2016-07-21 10:09

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit y

Extracting  NGIDS/NGIDS-DS-v1/feature_descr.csv                            0%  OK 

Would you like to replace the existing file NGIDS/NGIDS-DS-v1/ground_truth.csv
172799122 bytes, modified on 2016-05-04 12:36
with a new one
172799122 bytes, modified on 2016-05-04 12:36

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit A

Extracting  NGIDS/NGIDS-DS-v1/ground_truth.csv                             0%  OK 
Extracting  NGIDS/NGIDS-DS-v1/host logs/1.csv                              0%  OK 
Extracting  NGIDS/NGIDS-DS-v1/host logs/10.csv                             0%  1%  OK 
Extracting  NGIDS/NGIDS-DS-v1/host logs/

In [ ]:
import pandas as pd

url = "/content/NGIDS/NGIDS-DS-v1/host logs/1.csv"

# Try reading the file with a different encoding, like 'latin-1' or 'iso-8859-1'
df = pd.read_csv(url, encoding='latin-1')
# If 'latin-1' doesn't work, try other encodings like 'iso-8859-1', 'cp1252', etc.

# عرض أول 5 صفوف من البيانات
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 9 columns):
 #   Column                     Non-Null Count    Dtype 
---  ------                     --------------    ----- 
 0   ï»¿11/03/2016              1000000 non-null  object
 1   2:45:01                    1000000 non-null  object
 2   1830                       1000000 non-null  int64 
 3   /sbin/upstart-dbus-bridge  1000000 non-null  object
 4   142                        1000000 non-null  int64 
 5   45354                      1000000 non-null  int64 
 6   normal                     1000000 non-null  object
 7   normal.1                   1000000 non-null  object
 8   0                          1000000 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 68.7+ MB


,ï»¿11/03/2016,2:45:01,1830,/sbin/upstart-dbus-bridge,142,45354,normal,normal.1,0
0,11/03/2016,2:45:06,1804,/bin/dbus-daemon,256,45352,normal,normal,0
1,11/03/2016,2:45:06,2133,/usr/lib/i386-linux-gnu/gconf/gconfd-2,168,45372,normal,normal,0
2,11/03/2016,2:45:35,4528,/usr/bin/python3.4,3,39459,normal,normal,0
3,11/03/2016,2:45:44,1847,/usr/bin/ibus-daemon,102,37263,normal,normal,0
4,11/03/2016,2:45:44,1907,/usr/lib/ibus/ibus-ui-gtk3,168,37896,normal,normal,0


In [ ]:
df.rename(columns={'ï»¿11/03/2016':'Date', 1:'Time', 2:'Unique_Identification', 3:'Execution_Path', 4:'System_Calls_Identifiers', 5:"Event's_Unique_Identification", 6:'attacks', 7:"Sub_Type_Attack", 8:"Label"}, inplace=True)
df.columns.tolist()

['Date',
 '2:45:01',
 '1830',
 '/sbin/upstart-dbus-bridge',
 '142',
 '45354',
 'normal',
 'normal.1',
 '0']

In [ ]:
for file

In [ ]:
!pip install scapy



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.3 MB/s eta 0:00:00


In [ ]:
from scapy.all import rdpcap

# تحميل ملف الـ pcap
packets = rdpcap("/content/NGIDS/NGIDS-DS-v1/NGIDS.pcap")

# عدد الحزم
print(f"عدد الحزم: {len(packets)}")

# عرض أول 5 حزم
for i, pkt in enumerate(packets[:5]):
    print(f"\n🔹 Packet {i+1}:\n")
    pkt.show()


KeyboardInterrupt: 

#class

In [2]:
import pandas as pd
import numpy as np
import os
import glob
import math
from sklearn.preprocessing import LabelEncoder


In [ ]:
class Preprocessing():
  def __init__(self):
    pass

  def __file_path_collection(self, folder_path):
    try:
      joined_list = []
      for file_name in os.listdir(folder_path):
        file_name = os.path.join(folder_path, file_name)
        if os.path.isfile(file_name) and file_name.endswith('.csv'):
          joined_list.append(file_name)
      return joined_list

    except Exception as e:
      return f"An error occurred: {e}"


  def rename_columns(self, folder_path):
    files = self.__file_path_collection(folder_path)
    try:
      for file in files:
        df = pd.read_csv(file)
        column_name = df.columns.tolist()
        df.rename(columns={column_name[0]:'Date', column_name[1]:'Time', column_name[2]:'Unique_Identification', column_name[3]:'Execution_Path', column_name[4]:'System_Calls_Identifiers', column_name[5]:"Event's_Unique_Identification", column_name[6]:'attacks', column_name[7]:"Sub_Type_Attack", column_name[8]:"Label"}, inplace=True)
        df.to_csv(file, index=False)
        return f"done file: {file}"

    except Exception as e:
      return f"An error occurred: {e}"


  def data_encoding(self, folder_path):
    files = self.__file_path_collection(folder_path)
    encoder = LabelEncoder()
    all_value = []
    try:
      for file in files:
        df = pd.read_csv(file)
        print(file)
        all_value.extend(df['attacks'].tolist())
      encoder.fit(all_value)

      for file in files:
        df = pd.read_csv(file)
        print(file, 1)
        df['attacks'] = encoder.transform(df['attacks'])
        df.to_csv(file, index=False)
      return f"done file: {file}"

    except Exception as e:
      return f"An error occurred: {e}"



  def file_collection(self, folder_path):
    try:
      file_paths = self.__file_path_collection(folder_path)
      if file_paths:
        output_folder = os.path.join(os.getcwd(),'files')
        os.makedirs(output_folder ,exist_ok=True)
        output_file = os.path.join(output_folder,"final.csv")

        for file in file_paths:

          first = True
          chunk = pd.read_csv(file, chunksize=10000)
          for part in chunk:
            part.to_csv(output_file, mode='a', header=first, index=False)
            first = False
          print(f"done file: {file}")

    except Exception as e:
      print(f"An error occurred: {e}")


a = Preprocessing()
#a.file_path_collection("/content/NGIDS/NGIDS-DS-v1/host logs")
#a.file_collection("/content/NGIDS/NGIDS-DS-v1/host logs")
#a.rename_columns( '/content/NGIDS/NGIDS-DS-v1/host logs')
a.data_encoding('/content/NGIDS/NGIDS-DS-v1/host logs')

/content/NGIDS/NGIDS-DS-v1/host logs/40.csv
/content/NGIDS/NGIDS-DS-v1/host logs/51.csv
/content/NGIDS/NGIDS-DS-v1/host logs/28.csv
/content/NGIDS/NGIDS-DS-v1/host logs/70.csv
/content/NGIDS/NGIDS-DS-v1/host logs/44.csv
/content/NGIDS/NGIDS-DS-v1/host logs/84.csv
/content/NGIDS/NGIDS-DS-v1/host logs/69.csv
/content/NGIDS/NGIDS-DS-v1/host logs/43.csv
/content/NGIDS/NGIDS-DS-v1/host logs/83.csv
/content/NGIDS/NGIDS-DS-v1/host logs/35.csv
/content/NGIDS/NGIDS-DS-v1/host logs/32.csv
/content/NGIDS/NGIDS-DS-v1/host logs/76.csv
/content/NGIDS/NGIDS-DS-v1/host logs/46.csv
/content/NGIDS/NGIDS-DS-v1/host logs/36.csv
/content/NGIDS/NGIDS-DS-v1/host logs/89.csv
/content/NGIDS/NGIDS-DS-v1/host logs/27.csv
/content/NGIDS/NGIDS-DS-v1/host logs/93.csv
/content/NGIDS/NGIDS-DS-v1/host logs/12.csv
/content/NGIDS/NGIDS-DS-v1/host logs/16.csv
/content/NGIDS/NGIDS-DS-v1/host logs/22.csv
/content/NGIDS/NGIDS-DS-v1/host logs/2.csv
/content/NGIDS/NGIDS-DS-v1/host logs/81.csv
/content/NGIDS/NGIDS-DS-v1/host l

In [ ]:
max(c)

ValueError: operands could not be broadcast together with shapes (7,) (8,) 

In [ ]:
unique_attacks = list(set([item for sub in c for item in sub]))
unique_attacks

['Exploits',
 'Generic',
 'Shellcode',
 'Worms',
 'Reconnaissance',
 'Denial of Service',
 'Backdoors',
 'normal']

In [ ]:
def data_encoding(self, folder_path):
    files = self.__file_path_collection(folder_path)
    encoder = LabelEncoder()
    max_count = 0
    max_file = ""
    names_attacks = []
    try:
      for file in files:
        df = pd.read_csv(file)
        names_attacks.append([df['attacks'].unique(), len(df['attacks'].unique()),file])

      for names_attack, count, file in names_attacks:
        if count > max_count:
          max_count = count
          max_file = file
          max_names_attacks = names_attack
      return max_names_attacks, max_count, max_file
    except Exception as e:
      return f"An error occurred: {e}"

'normal'

In [ ]:
from scapy.all import rdpcap
from datetime import datetime

packets = rdpcap("NGIDS.pcap")

# تحويل الحزم إلى بيانات قابلة للتحليل
network_data = []
for pkt in packets:
    if pkt.time and hasattr(pkt, "summary"):
        network_data.append({
            "timestamp": datetime.fromtimestamp(pkt.time),
            "summary": pkt.summary()
        })

network_df = pd.DataFrame(network_data)


,Unnamed: 0,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,0,-122.05,37.37,27.0,3885.0,661.0,1537.0,606.0,6.6085,344700.0
1,1,-118.30,34.26,43.0,1510.0,310.0,809.0,277.0,3.5990,176500.0
2,2,-117.81,33.78,27.0,3589.0,507.0,1484.0,495.0,5.7934,270500.0
3,3,-118.36,33.82,28.0,67.0,15.0,49.0,11.0,6.1359,330000.0
4,4,-119.67,36.33,19.0,1241.0,244.0,850.0,237.0,2.9375,81700.0
...,...,...,...,...,...,...,...,...,...,...
19995,19995,-124.26,40.58,52.0,2217.0,394.0,907.0,369.0,2.3571,111400.0
19996,19996,-124.27,40.69,36.0,2349.0,528.0,1194.0,465.0,2.5179,79000.0
19997,19997,-124.30,41.84,17.0,2677.0,531.0,1244.0,456.0,3.0313,103600.0
19998,19998,-124.30,41.80,19.0,2672.0,552.0,1298.0,478.0,1.9797,85800.0
